In [47]:
from fairness.matching import compute
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
dataset = pd.read_csv('./dataset/adult.data', sep=',')

In [54]:
dataset

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32556,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32557,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0
32558,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0


In [3]:
dataset.iloc[:, len(dataset.columns) - 1:]

,<=50K
0,<=50K
1,<=50K
2,<=50K
3,<=50K
4,<=50K
...,...
32555,<=50K
32556,>50K
32557,<=50K
32558,<=50K


In [4]:
columns_to_drop = ['39']
protected_attributes = [' White', ' Never-married', ' Male']
output_column = ' <=50K'

In [5]:
fair_dataset = compute(dataset, protected_attributes, output_column, columns_to_drop)

        Attribute  Disparate Impact
0           White          0.596327
1   Never-married          7.151484
2            Male          0.358006
1
Begun returning apriori dataset format
Finished on returning apriori dataset format
Begun association rule mining
Finished association rule mining
                                              Antecedent   
0                                                     []  \
1                                                     []   
2                                                     []   
3                                                     []   
4                                             [ 13 = 10]   
...                                                  ...   
10797  [ United-States = 39,  13 = 9,  Not-in-family ...   
10798  [ United-States = 39,  13 = 9,  Not-in-family ...   
10799  [ United-States = 39,  13 = 9,  Not-in-family ...   
10800  [ United-States = 39,  13 = 9,  Not-in-family ...   
10801  [ United-States = 39,  Not-in-family = 0

In [6]:
fair_dataset

,39,State-gov,77516,Bachelors,13,Adm-clerical,Not-in-family,2174,0,40,United-States,<=50K
0,50,6,83311,9,13,4,0,0,0,13,39,0
1,38,4,215646,11,9,6,1,0,0,40,39,0
2,53,4,234721,1,7,6,0,0,0,40,39,0
3,28,4,338409,9,13,10,5,0,0,40,5,0
4,37,4,284582,12,14,4,5,0,0,40,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,4,257302,7,12,13,5,0,0,38,39,0
32556,40,4,154374,11,9,7,0,0,0,40,39,1
32557,58,4,151910,11,9,1,4,0,0,40,39,0
32558,22,4,201490,11,9,1,3,0,0,20,39,0


In [11]:
fair_dataset.rename(columns={" <=50K": "50K"}, inplace=True)

In [40]:
output_column = "50K"

In [12]:
fair_dataset

,39,State-gov,77516,Bachelors,13,Adm-clerical,Not-in-family,2174,0,40,United-States,50K
0,50,6,83311,9,13,4,0,0,0,13,39,0
1,38,4,215646,11,9,6,1,0,0,40,39,0
2,53,4,234721,1,7,6,0,0,0,40,39,0
3,28,4,338409,9,13,10,5,0,0,40,5,0
4,37,4,284582,12,14,4,5,0,0,40,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32555,27,4,257302,7,12,13,5,0,0,38,39,0
32556,40,4,154374,11,9,7,0,0,0,40,39,1
32557,58,4,151910,11,9,1,4,0,0,40,39,0
32558,22,4,201490,11,9,1,3,0,0,20,39,0


In [41]:
X = fair_dataset.loc[:,:output_column]
y = fair_dataset.loc[:, output_column:]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
standard_scaler_train = StandardScaler()
X_train = standard_scaler_train.fit_transform(X_train)

standard_scaler_test = StandardScaler()
X_test = standard_scaler_test.fit_transform(X_test)

In [51]:
model = GaussianNB()
model.fit(X_train, y_train)

/home/antonio/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [52]:
y_pred = model.predict(X)
fair_dataset['y_pred'] = y_pred

/home/antonio/.local/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but GaussianNB was fitted without feature names
  warnings.warn(


In [53]:
print(confusion_matrix(y, y_pred))
print(accuracy_score(y, y_pred))
print(classification_report(y, y_pred))

[[24705    14]
 [ 3388  4453]]
0.8955159705159705
              precision    recall  f1-score   support

           0       0.88      1.00      0.94     24719
           1       1.00      0.57      0.72      7841

    accuracy                           0.90     32560
   macro avg       0.94      0.78      0.83     32560
weighted avg       0.91      0.90      0.88     32560

